In [4]:

from __future__ import print_function, division
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.nn import init
import torch.nn.functional as F
import functools
import numpy as np
import torchvision
from torchvision import transforms, datasets, models
import os
import time

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Training settings
batch_size = 50

In [6]:
#%%

# p = Augmentor.Pipeline()
# p.random_erasing(0.5,0.4)




train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop((32, 32), padding=4),   #left, top, right, bottom
#     p.torch_transform(),
#     p.sample(10000),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                               train=True,
                               transform=train_transform,
                               download=True)

test_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                              train=False,
                              transform=test_transform)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, # 64
                                           shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

dataloaders = {
    'train': train_loader,
    'test' : test_loader    
}
image_datasets = {
    'train': train_dataset,
    'test' : test_dataset
}
dataset_sizes = {
    'train': len(image_datasets['train']),
    'test' : len(image_datasets['test'])
}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print('train_size: ',dataset_sizes['train'],'\ntest_size : ',dataset_sizes['test'])
print(classes)
print('len class: ', len(classes))



Files already downloaded and verified
train_size:  50000 
test_size :  10000
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
len class:  10


In [38]:
import cv2
# # ADDING NOISE
noise = []
noise = np.random.rand(32, 32,3)
import pickle

# pickle_out = open("batch1","wb")
data = []
filenames = []
occuldes = []
for n in range(2):
    inputs, labels = next(iter(dataloaders['train']))
    pickle_out = open("batch{}".format(n+1),"wb")
    data = []
    for m in range(batch_size):
        if(m<35):
            images_ = inputs[m].numpy()
            class_  = labels[m]
            images_ = np.transpose(images_, [1,2,0])
            
            r = images_[:,:,0]
            g = images_[:,:,1]
            b = images_[:,:,2]

            images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
            images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
            images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            data.append(images_)
            filename = '{}_{}.png'.format(classes[class_.numpy()],n*m+m)
            filenames.append(filename)
#         print(classes[class_.numpy()])
            plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occulde/{}_{}.png'.format(classes[class_.numpy()],n*m+m), images_)
        
            occulde = 0
            occuldes.append(occulde)
    #         plt.imshow(images_)
#         plt.show()
        else:
            images_ = inputs[m].numpy()
            class_  = labels[m]
            images_ = np.transpose(images_, [1,2,0])
            x_random = np.random.randint(6, 27, size=1, dtype='uint8')
            y_random = np.random.randint(6, 27, size=1, dtype='uint8')
            w_random = np.random.randint(8, 13, size=1, dtype='uint8')
            h_random = np.random.randint(8, 13, size=1, dtype='uint8')
            images_[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]=noise[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]

            r = images_[:,:,0]
            g = images_[:,:,1]
            b = images_[:,:,2]

            images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
            images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
            images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            data.append(images_)
            filename = '{}_{}.png'.format(classes[class_.numpy()],n*m+m)
            filenames.append(filename)
            
            plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occulde/{}_{}.png'.format(classes[class_.numpy()],n*m+m), images_)
 
            occulde = 1
            occuldes.append(occulde)
        
    pickle_cifar10 = {'labels': labels.numpy(),'data': data, 'filenames': filenames, 'occuldes': occuldes}
    pickle.dump(pickle_cifar10, pickle_out)
    pickle_out.close()
    data.clear()
    filenames.clear()
    occuldes.clear()



In [7]:
def unpickle(file):
#     import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [39]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch1")
print(pickle_in)

{'labels': array([3, 2, 1, 9, 0, 0, 3, 3, 7, 2, 9, 7, 8, 1, 3, 2, 1, 9, 4, 1, 0, 5,
       0, 6, 9, 7, 2, 6, 3, 7, 2, 5, 6, 2, 5, 6, 0, 5, 0, 2, 1, 6, 4, 2,
       5, 0, 9, 6, 8, 7]), 'data': [array([[[0.75652176, 0.8640777 , 0.9245283 ],
        [0.45217392, 0.35436895, 0.3537736 ],
        [0.4       , 0.15533979, 0.09905661],
        ...,
        [0.33478263, 0.20873788, 0.23584908],
        [0.26956522, 0.11650486, 0.15566036],
        [0.3826087 , 0.2524272 , 0.26415095]],

       [[0.7521739 , 0.8592233 , 0.9198113 ],
        [0.45652175, 0.37378642, 0.36792454],
        [0.37826088, 0.16019416, 0.09905661],
        ...,
        [0.3826087 , 0.30097088, 0.33490568],
        [0.22608696, 0.13106796, 0.16509433],
        [0.32608697, 0.21359226, 0.22641511]],

       [[0.7434783 , 0.82524276, 0.8537736 ],
        [0.47391304, 0.38834953, 0.36792454],
        [0.36956522, 0.16019416, 0.09905661],
        ...,
        [0.40869567, 0.3592233 , 0.39150944],
        [0.20434783, 0.15533

In [40]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch2")
print(pickle_in)

{'labels': array([8, 5, 8, 8, 8, 7, 2, 3, 5, 8, 2, 0, 7, 8, 2, 1, 0, 4, 8, 3, 6, 9,
       9, 8, 5, 8, 1, 9, 7, 6, 4, 1, 3, 9, 7, 4, 9, 7, 7, 7, 3, 9, 7, 1,
       3, 3, 1, 0, 3, 3]), 'data': [array([[[0.85375494, 0.92800003, 0.9591837 ],
        [0.86956525, 0.944     , 0.97959185],
        [0.8656127 , 0.94      , 0.9755102 ],
        ...,
        [0.92885375, 0.984     , 0.9877551 ],
        [0.92885375, 0.984     , 0.9918367 ],
        [0.9209486 , 0.98      , 1.        ]],

       [[0.8577075 , 0.916     , 0.94285715],
        [0.8735178 , 0.932     , 0.9632653 ],
        [0.8774704 , 0.936     , 0.96734697],
        ...,
        [0.91699606, 0.972     , 0.9714286 ],
        [0.9130435 , 0.972     , 0.97959185],
        [0.9051384 , 0.964     , 0.97959185]],

       [[0.8458498 , 0.916     , 0.9346939 ],
        [0.86166006, 0.932     , 0.9510204 ],
        [0.86956525, 0.94      , 0.9591837 ],
        ...,
        [0.91699606, 0.972     , 0.9755102 ],
        [0.9130435 , 0.968  

In [10]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/data/cifar-10-batches-py/data_batch_1")
print(pickle_in)

{b'batch_label': b'training batch 1 of 5', b'labels': [6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3, 6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4, 0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5, 6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2, 1, 1, 2, 1, 5, 9, 9, 0, 8, 4, 1, 1, 6, 3, 

In [11]:
pickle1_out = open("batch1_test","wb")

pickle.dump(pickle_in, pickle1_out)
pickle1_out.close()

In [15]:
dfd = unpickle("/home/lee/Research/11.project_tangqing/batch1_test")
print(dfd)

{b'batch_label': b'training batch 1 of 5', b'labels': [6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3, 6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4, 0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5, 6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2, 1, 1, 2, 1, 5, 9, 9, 0, 8, 4, 1, 1, 6, 3, 